In [1]:
using Iterators
using DataStructures
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")

3-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              

In [2]:
using DataStructures

function Base.sum(acc::Accumulator)
    sum(values(acc.map))
end

function freq2prob{T,V<:Number}(acc::Accumulator{T,V})
    ret=Dict{T,Float64}()
    total = sum(acc)
    for (k,v) in acc
        ret[k]=v/total
    end
    ret
end

freq2prob (generic function with 1 method)

In [3]:
function clean(ss)
    @pipe (ss 
    |> replace(_, r"\[.*?\] ?","")  #Remove Nonword sounds
    |> replace(_, r"\<.*?\> ","")  #Remove Verbal Deltions
    |> replace(_, r"\*(.*?)\*",s"\1") #Remove mispronounciation marks
    |> replace(_,r"\:|\-\s\.\s\-", "") #remove intraword pauses
    |> replace(_,r"\w+\- ","") #remove stuttered words
    |> replace(_, r"[!\.,\?]","")        #Remove punctation as it is not used traditionally (see sro spec)
    
    |> replace(_, r"\s+",' ') #Remove repeated spaces
    #|> replace(_, r"([A-Z])\s([A-Z])", s"\1\2") #Merge len(2) abbrev
    |> replace(_, r".*[~\(\)\-\<\>#'].*","")#Remove everything if anything unfixable found
    |> lowercase
    |> strip 
    )
end


clean (generic function with 1 method)

In [4]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

function collect_grams_stats(sentences)
    unigrams = counter(AbstractString)
    bigrams = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        push!(bigrams[START_MARKER], sent[1])
        for ii in 1:length(sent)-1
            push!(unigrams,sent[ii])
            push!(bigrams[sent[ii]], sent[ii+1])    
        end
        push!(unigrams,sent[end])
        push!(bigrams[sent[end]], END_MARKER)
    end
    
    
    [k=>v.map for (k,v) in bigrams], unigrams.map
end




collect_grams_stats (generic function with 1 method)

In [5]:
path="../../Resources/corpora/atis2_text/"
function valid(ss)
    typeof(ss) <: ASCIIString  && length(ss)>0
end
    
corpus = @pipe readdir(path) |> filter!(fn -> splitext(fn)[2]==".sro", _) |> map(_) do fn
    try open(readall, path*fn) end
    end |> filter!(valid,_) |> map(clean,_) |> filter!(valid,_) |> map(s->split(s),_);
corpus_vocab = @pipe corpus |> map(Set,_) |> reduce(union,_)
length(corpus_vocab)

1131

In [6]:
using WordEmbeddings
LL, word_indexes, indexed_words = load_word2vec_embeddings("word_emb_data/GoogleNews-vectors-negative300.bin", length(corpus_vocab), corpus_vocab);

In [7]:
setdiff(corpus_vocab,indexed_words)


16-element Array{UTF8String,1}:
 "and"      
 "a"        
 "respeak"  
 "nondirect"
 "maluso"   
 "of"       
 "panam"    
 "stapleton"
 "fokker"   
 "lufthansa"
 "to"       
 "laguardia"
 "nonjets"  
 "hartfield"
 "mcdonnell"
 "dulles"   

In [8]:
#Kind of the opposite of a stop word. This word has little meaning (So zero value), but much structural importance
forcewords = ["and", "a", "of", "to"]
for word in forcewords
    @assert(!(word in indexed_words))
    push!(indexed_words, word)
    word_indexes[word] = length(indexed_words)
end
LL = [LL zeros(size(LL,1),length(forcewords))]

300x1119 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  -0.0548293   0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299        0.0239878   0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       -0.0190492   0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772        0.0826471   0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199        0.0620861   0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …   0.0465646   0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       -0.0786156   0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       -0.0516041   0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369         0.00902063  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522        0.0364857   0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …   0.0395094   0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       -0.0588609   0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       -0.0903071   0.0  0.0  0.0  0.

In [9]:
@assert(!(START_MARKER in indexed_words))
push!(indexed_words, START_MARKER)
word_indexes[START_MARKER] = length(indexed_words)

@assert(!(END_MARKER in indexed_words))
push!(indexed_words, END_MARKER)
word_indexes[END_MARKER] = length(indexed_words)
LL = [LL zeros(size(LL,1),2)]

300x1121 Array{Float64,2}:
  0.0529562  -0.00851202  -0.0123606    …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0654598  -0.0342245   -0.0222299       0.0  0.0  0.0  0.0  0.0  0.0
  0.0661953   0.0322839    0.0655398       0.0  0.0  0.0  0.0  0.0  0.0
  0.0470722   0.0458679    0.0394772       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207  -0.0131429   -0.0866199       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0820086  -0.0462207    0.0249128    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0614145  -0.00094823  -0.0111628       0.0  0.0  0.0  0.0  0.0  0.0
 -0.11621    -0.0522188   -0.0705224       0.0  0.0  0.0  0.0  0.0  0.0
  0.0156294   0.0465735    0.092369        0.0  0.0  0.0  0.0  0.0  0.0
  0.0992929   0.0624509    0.0927522       0.0  0.0  0.0  0.0  0.0  0.0
 -0.0856861  -0.122785    -0.0563412    …  0.0  0.0  0.0  0.0  0.0  0.0
 -0.028133   -0.0287556   -0.0605572       0.0  0.0  0.0  0.0  0.0  0.0
  0.0522207   0.0515131   -0.0540416       0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                  

In [ ]:
known_vocab

In [10]:
known_vocab = Set(indexed_words)
known_corpus = filter(corpus) do sent
    for word in sent
        if !( word in known_vocab)
            return false
        end
    end
    true
end;

In [11]:
using StatsBase
# modified from https://github.com/JoFrhwld/GoodTuring.jl/blob/master/GoodTuring.jl
function simpleGoodTuring(speciesCountDict::Dict)
    speciesCountVec = collect(values(speciesCountDict))
        
    totalCounts = sum(speciesCountVec)
    cofcDict = countmap(speciesCountVec)
    r = sort(collect(keys(cofcDict)))

    N = size(r,1)
    Nr = [cofcDict[r[i]] for i in 1:N]

    p0 = haskey(cofcDict, 1.0) ? cofcDict[1.0] / totalCounts : 0.0
    
    Z = sgtZ(r,Nr)
    logr = map(log,r)
    logZ = map(log,Z)

    X = hcat(ones(N), logr)
    Y = copy(logZ )
    coefs = X\Y
    intercept = coefs[1]
    slope = coefs[2]

    useY = false
    rSmooth = Array{Float64}(N)
    for i in 1:N
        @inbounds thisr = r[i]
        
        #y = ((thisr+1.0)^(slope+1.0))/(thisr^slope)
        #The above is the much simplified form of the below (Performance identical output differs by 10^-16)
        y = (thisr+1.0) * exp(slope * log(thisr+1.0) + intercept) / exp(slope * log(thisr) + intercept)

        if !in(thisr+1, r)
            useY = true
        end

        if useY
            rSmooth[i] = y
        else
            x = (thisr+1) * cofcDict[thisr + 1]/cofcDict[thisr]
            thisNr = cofcDict[thisr]
            thisNr1 = cofcDict[thisr+1]

            t = 1.96 * ((thisr+1)^2) * (thisNr1 / thisNr^2) * (1 + (thisNr1 / thisNr))

            if abs(x-y) > t
                @inbounds rSmooth[i] = x
            else
                useY = true
                @inbounds rSmooth[i] = y
            end
        end
    end

    smoothTot = sum(Nr.*rSmooth)
    sgtProb  = (1.0 - p0) .* (rSmooth/smoothTot)
    sgtProbDict = Dict([r[i] => sgtProb[i] for i in 1:N])
    sgtDict = Dict([sp=>sgtProbDict[speciesCountDict[sp]] for sp in keys(speciesCountDict)])

    sgtDict, sgtProbDict, p0
end


function sgtZ(r::Array, Nr::Array)
    j = r
    i = [0; j[1:end-1]]
    lastK = 2*j[end] - i[end]
    k = [j[2:end]; lastK]
    Float64[(2*Nr[iter])/(k[iter]-i[iter]) for iter = 1:length(j)]
end


sgtZ (generic function with 1 method)

In [12]:

function katz_bigrams(bigram_freq, unigrams_freq)
    k_bigrams = Dict()
    
    for first in keys(bigram_freq)
        smoothed,_,p0 = simpleGoodTuring(bigram_freq[first])
        k_bigrams[first] = smoothed
        
        backoff_keys = setdiff(keys(unigram_freq),keys(smoothed))
        #share the p0 proability mass between them
        total = sum([unigrams_freq[key] for key in backoff_keys])
        for second in backoff_keys
            k_bigrams[first][second]=p0.*unigrams_freq[second]./total
        end
    end
    k_bigrams
end

katz_bigrams (generic function with 1 method)

In [22]:
function bigrams_dict2mat(bigrams::Dict, word_indexes::Dict{AbstractString,Int64}, dense=False)
    mat  = (dense ? zeros: spzeros)(length(word_indexes),length(word_indexes))
    for first in keys(bigrams)
        for second in keys(bigrams[first])
            mat[word_indexes[second], word_indexes[first]] = bigrams[first][second]
        end
    end
    mat
end

bigrams_dict2mat (generic function with 2 methods)

In [24]:
bigram_freq, unigram_freq = collect_grams_stats(known_corpus);
kbigrams=katz_bigrams(bigram_freq, unigram_freq)
kbigrams_mat = bigrams_dict2mat(kbigrams,word_indexes,true)


1121x1121 Array{Float64,2}:
 0.000738563  0.000845547  0.00104524   …  0.000216622  0.00248123   0.0
 0.000738563  0.00254699   0.000843736     0.000144551  0.000804161  0.0
 0.00293742   0.0166282    0.000649407     0.000106928  0.000424392  0.0
 0.000462381  0.00131654   0.0175479       0.000337286  0.0342978    0.0
 0.000642955  0.00183069   0.00166171      0.000469007  0.00728763   0.0
 6.13763e-5   0.000174757  0.00021603   …  4.47713e-5   0.000198714  0.0
 5.61369e-7   1.59839e-6   1.97588e-6      4.09494e-7   8.01806e-7   0.0
 2.24548e-6   6.39355e-6   0.00166171      1.63798e-6   3.20723e-6   0.0
 0.243514     0.0948053    0.00459815      0.00608653   0.00522763   0.0
 0.00108937   0.000565209  0.000843736     5.74656e-5   0.000500169  0.0
 5.42657e-6   1.54511e-5   1.91002e-5   …  3.95844e-6   0.000125023  0.0
 1.02918e-5   2.93038e-5   3.62245e-5      7.50739e-6   1.46998e-5   0.0
 2.95654e-5   8.41817e-5   0.000104063     2.15667e-5   0.000273493  0.0
 ⋮                     

In [25]:
function collect_cooccur_stats(sentences)
    unioccur = counter(AbstractString)
    bioccur = DefaultDict(()->counter(AbstractString))
    
    for sent in sentences
        for ii in 1:length(sent)
            push!(unioccur, sent[ii])
            for jj in 1:length(sent)
                if ii==jj
                    continue
                end
                push!(bioccur[sent[ii]], sent[jj])    
            end
        end       
    end
    
    [k=>v.map for (k,v) in bioccur], unioccur.map
    
end

collect_cooccur_stats (generic function with 1 method)

In [26]:
bioccur_freq, unioccur_freq = collect_cooccur_stats(known_corpus)
kbioccur=katz_bigrams(bioccur_freq, unioccur_freq)
kbioccur_mat = bigrams_dict2mat(kbioccur,word_indexes,true)


1121x1121 Array{Float64,2}:
 0.00854599   0.00662352   0.0205733    …  0.0108031    0.010797     0.0  0.0
 0.0040545    0.00388946   0.00562277      0.00596491   0.00500017   0.0  0.0
 0.0121497    0.00542734   0.00420745      0.00832641   0.00584526   0.0  0.0
 0.0136121    0.0200386    0.0255273       0.0307899    0.0126852    0.0  0.0
 0.017268     0.0131174    0.0223426       0.02647      0.030221     0.0  0.0
 0.0068225    0.00073039   0.000407736  …  0.00141513   0.00330998   0.0  0.0
 1.03952e-5   1.01867e-5   0.000147622     1.50007e-5   7.41077e-6   0.0  0.0
 0.000140777  0.000110425  0.000407736     9.62274e-5   6.2145e-5    0.0  0.0
 0.0511115    0.0652404    0.0455207       0.057919     0.0389492    0.0  0.0
 0.00499457   0.00252264   0.00765738      0.00354587   0.00295346   0.0  0.0
 0.000347873  8.81605e-5   0.000759593  …  9.62274e-5   0.000180678  0.0  0.0
 0.00149562   0.000390776  0.000233394     4.82865e-5   0.000827552  0.0  0.0
 0.00113016   0.00107126   0.0005834

In [27]:
open("atis_data.jsz","w") do fh
    data = Dict([
        ("bigrams", kbigrams_mat),
        ("bioccur", kbioccur_mat),
        ("LL",LL),
        ("word_indexes", word_indexes),
        ("indexed_words", indexed_words),
        ])
    serialize(fh, data)    
end

In [40]:
function likelyhood(sent, bigrams)
    words = split(sent)
    words = [START_MARKER; words; END_MARKER]
    
    p=1.0
    for ii in 1:length(words)-1
        p*=bigrams[words[ii]][words[ii+1]]
    end
    p
end


function select_word{S<:AbstractString,V}(unigrams::Dict{S,V})
    cutoff = rand()
    total = 0.0
    for next_word in keys(unigrams)
        total+=unigrams[next_word]
        if total>=cutoff
            return next_word
        end
    end
    assert(False, "Should never reach here") 
end

function random_walk(bigrams)
    words=[]
    cur = START_MARKER
    while(cur!=END_MARKER)
        cur = select_word(bigrams[cur])
        push!(words,cur)
    end
    words = words[1:end-1]
    join(words, " ")
end

walk =random_walk(kbigrams) 
print(walk*"\t")
print(likelyhood(walk,kbigrams))

show the fare for one way	3.6139704065231135e-8